In [1]:
# Expedia Hotel Recommendation
# Submission 1
# 5/24/2016

In [2]:
#importing pandas for data frame operations
#Reading csv files 
import pandas as pd

destinations = pd.read_csv("destinations.csv")

In [3]:
#Loading the entire train dataset into memory

import pandas as pd
import numpy as np

train = pd.read_csv('train.csv',
                    dtype={'is_booking':bool,'srch_destination_id':np.int32, 'hotel_cluster':np.int32},
                    usecols=['srch_destination_id','is_booking','hotel_cluster'],
                    chunksize=1000000)

In [4]:
df=pd.DataFrame()

In [5]:
df = pd.concat(chunk for chunk in train)

In [6]:
df.shape

(37670293, 3)

In [7]:
df.head(5)

,srch_destination_id,is_booking,hotel_cluster
0,8250,False,1
1,8250,True,1
2,8250,False,1
3,14984,False,80
4,14984,False,21


In [8]:
train = df
train.dtypes

srch_destination_id    int32
is_booking              bool
hotel_cluster          int32
dtype: object

In [9]:
#Loading test dataset into memory
test = pd.read_csv("test.csv")

In [10]:
test.shape

(2528243, 22)

In [11]:
#Algorithm for finding suitable clusters

def make_key(items):
    return "_".join([str(i) for i in items])

match_cols = ["srch_destination_id"]
cluster_cols = match_cols + ['hotel_cluster']
groups = train.groupby(cluster_cols)
top_clusters = {}
for name, group in groups:
    clicks = len(group.is_booking[group.is_booking == False])
    bookings = len(group.is_booking[group.is_booking == True])
    
    score = bookings + .086 * clicks    
    
    clus_name = make_key(name[:len(match_cols)])
    if clus_name not in top_clusters:
        top_clusters[clus_name] = {}
    top_clusters[clus_name][name[-1]] = score
   

In [12]:
top_clusters

{'50088': {3: 0.172, 20: 0.086},
 '44884': {7: 0.42999999999999994,
  13: 1.086,
  14: 2.4299999999999997,
  39: 4.688,
  91: 1.6019999999999999},
 '11542': {91: 28.728},
 '11543': {20: 0.086,
  27: 34.512,
  53: 0.42999999999999994,
  57: 0.258,
  60: 0.516,
  74: 1.688},
 '11540': {53: 0.172, 93: 0.344},
 '11541': {94: 4.602},
 '11547': {3: 0.258,
  6: 13.707999999999998,
  7: 3.6979999999999995,
  8: 29.296,
  10: 12.234,
  13: 20.759999999999998,
  14: 0.42999999999999994,
  15: 23.921999999999997,
  16: 2.666,
  17: 22.145999999999997,
  20: 0.344,
  27: 13.072,
  30: 4.3,
  32: 14.707999999999998,
  33: 13.008,
  35: 1.1179999999999999,
  39: 10.536,
  40: 3.526,
  44: 0.946,
  47: 19.113999999999997,
  48: 33.006,
  50: 94.59799999999998,
  72: 11.084,
  75: 28.177999999999997,
  77: 4.6979999999999995,
  80: 298.876,
  83: 8.224,
  85: 1.172,
  87: 1.8599999999999999,
  89: 1.376,
  90: 0.172,
  93: 17.018},
 '11544': {3: 1.3439999999999999,
  20: 0.172,
  44: 0.086,
  53: 0.17

In [13]:
#To find the top 5 clusters for each search destination

import operator

cluster_dict = {}
for n in top_clusters:
    tc = top_clusters[n]
    top = [l[0] for l in sorted(tc.items(), key=operator.itemgetter(1), reverse=True)[:5]]
    cluster_dict[n] = top

In [14]:
cluster_dict

{'13357': [5, 15],
 '11542': [91],
 '11543': [27, 74, 60, 53, 57],
 '11540': [93, 53],
 '11541': [94],
 '11547': [80, 50, 48, 8, 75],
 '11544': [3, 20, 53, 44, 61],
 '11545': [30, 20, 44],
 '11548': [5],
 '11549': [35],
 '5988': [42, 15],
 '5989': [78, 30, 44, 67, 62],
 '19399': [62, 53],
 '19398': [90, 85],
 '5982': [81, 82, 30, 67],
 '5983': [91, 93, 7],
 '5980': [42, 82, 32, 48, 91],
 '5981': [8],
 '5986': [48, 3, 33, 71, 26],
 '5987': [83, 93, 94, 40, 73],
 '5984': [48, 91],
 '5985': [13, 48, 23, 40, 76],
 '57481': [20],
 '57480': [78, 3, 53],
 '57483': [47],
 '17541': [5, 44, 89],
 '57482': [60, 3, 20],
 '57485': [86, 38],
 '36709': [76, 60],
 '46139': [44, 89],
 '46138': [60, 3],
 '46137': [3, 44, 30, 53, 93],
 '46136': [53, 3, 44, 60, 20],
 '46135': [53, 30, 44, 85, 60],
 '46134': [32],
 '46133': [28, 32, 15, 42, 14],
 '46132': [20, 44, 53, 78, 76],
 '46131': [3],
 '46130': [63, 30, 58, 61, 78],
 '57489': [48, 17, 91, 10, 93],
 '53631': [51, 48, 77, 33, 6],
 '53909': [82, 62, 36

In [15]:
#Making predictions based on top 5 clusters for each destination

preds = []
for index, row in test.iterrows():
    key = make_key([row[m] for m in match_cols])
    if key in cluster_dict:
        preds.append(cluster_dict[key])
    else:
        preds.append([])

In [16]:
preds

[[5, 37, 55, 11, 22],
 [5],
 [0, 31, 96, 91, 77],
 [1, 45, 79, 24, 54],
 [91, 42, 2, 48, 59],
 [91, 42, 28, 95, 4],
 [95, 21, 2, 33, 98],
 [95, 18, 91, 98, 68],
 [1, 45, 79, 24, 54],
 [55, 32, 10, 34, 50],
 [33, 18, 4, 19, 21],
 [25, 38, 6, 75, 82],
 [0, 31, 96, 91, 77],
 [9, 6, 17, 37, 96],
 [95, 28, 91, 72, 83],
 [48, 47, 42, 17, 77],
 [71, 34, 77, 0, 18],
 [6, 91, 18, 48, 94],
 [59, 21, 42, 25, 19],
 [95, 91, 59, 68, 9],
 [91, 18, 42, 4, 48],
 [83, 56, 41, 77, 17],
 [9, 6, 17, 37, 96],
 [65, 66, 87, 61, 8],
 [56, 70, 98, 41, 69],
 [56, 70, 98, 41, 69],
 [99, 9, 2, 46, 6],
 [48, 73, 39, 40, 14],
 [91, 7, 43, 48, 47],
 [65, 66, 52, 87, 96],
 [64, 2, 29, 59, 5],
 [46, 12, 36, 29, 81],
 [28, 48, 6, 42, 16],
 [1, 45, 79, 24, 54],
 [19, 70, 56, 41, 37],
 [91, 94, 48, 32, 13],
 [42, 91, 15, 48, 51],
 [34, 72, 99, 55, 17],
 [2, 68, 9, 59, 98],
 [19, 70, 56, 41, 37],
 [63, 58, 26, 62, 81],
 [46, 36, 29, 12, 62],
 [34, 18, 70, 42, 91],
 [1, 45, 79, 24, 54],
 [55, 9, 21, 95, 49],
 [91, 59, 92,

In [24]:
#Calculate error
#Implementation from ml_metrics package by Ben Hamner
#Error calculation is useful only while building the model and not for the entire test dataset

import numpy as np

def apk(actual, predicted, k=10):
    if len(predicted)>k:
        predicted = predicted[:k]

    score = 0.0
    num_hits = 0.0
    
    for i,p in enumerate(predicted):
        if p in actual and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i+1.0)

    if not actual:
        return 0.0

    return score / min(len(actual), k)


def mapk(actual, predicted, k=10):    
    return np.mean([apk(a,p,k) for a,p in zip(actual, predicted)])

mapk([[l] for l in test["hotel_cluster"]], preds, k=5)

In [17]:
#Making a submission file

write_p = [" ".join([str(l) for l in p]) for p in preds]
write_frame = ["{0},{1}".format(test["id"][i], write_p[i]) for i in range(len(preds))]
write_frame = ["id,hotel_cluster"] + write_frame
with open("predictions.csv", "w+") as f:
    f.write("\n".join(write_frame))

In [19]:
predictions = pd.read_csv('predictions.csv')

In [20]:
predictions.shape

(2528243, 2)

In [21]:
predictions.head(5)

,id,hotel_cluster
0,0,5 37 55 11 22
1,1,5
2,2,0 31 96 91 77
3,3,1 45 79 24 54
4,4,91 42 2 48 59
